# Adapter

Benchmarking example

In [1]:
import time
import pathlib
import csv
import itertools
from datetime import datetime
from tqdm import tqdm
import torch
import os
from datasets import load_dataset
from collections import namedtuple

import threading
import pynvml
from utils.adapter import BERTAdapter

ImportError: cannot import name 'AdapterTrainer' from 'transformers' (/Users/blake/miniforge3/envs/fast/lib/python3.9/site-packages/transformers/__init__.py)

In [ ]:
device_name = "cpu"  # default device is CPU
if torch.cuda.is_available():
    # I read that this works for detecting if notebook is being run in a colab environment, not sure though
    if 'COLAB_GPU' in os.environ:
        print("colab environment")
        device_name = "gpu" 
    else:
        device_name = "cuda" # CUDA for NVIDIA GPU
elif torch.backends.mps.is_available():
    device_name = torch.device("mps")  # Metal Performance Shaders for Apple M-series GPU

device = torch.device(device_name)
print(device_name)

In [ ]:
task_param = "cola"

In [ ]:
# sentence_type: ["one", "two"]
# class_type: ["BC", "MC", "R"]
# input_size: int (represents input size of feedforward, aka embedding size)
# col_names: column names of relavent sentences on hugging face
# num_classes needs to be 1 for BC and R, only change value to number of classes for MC

TaskConfig = namedtuple("TaskConfig", ["sentence_type", "class_type", "num_classes", "input_size", "col_names"])

task_configs = {
    "cola": TaskConfig("one", "BC", 1, 768, ['sentence']),
    "sst2": TaskConfig("one", "BC", 1, 768, ['sentence']),
    "mrpc": TaskConfig("two", "BC", 1, 768, ['sentence1', 'sentence2']),
    "stsb": TaskConfig("two", "R", 1, 768, ['sentence1', 'sentence2']),
    "qqp": TaskConfig("two", "BC", 1, 768, ['question1', 'question2']),
    "mnli_matched": TaskConfig("two", "MC", 3, 768, ['premise', 'hypothesis']),
    "mnli_mismatched": TaskConfig("two", "MC", 3, 768, ['premise', 'hypothesis']),
    "qnli": TaskConfig("two", "BC", 1, 768, ['question', 'sentence']),
    "rte": TaskConfig("two", "BC", 1, 768, ['sentence1', 'sentence2']),
    "wnli": TaskConfig("two", "BC", 1, 768, ['sentence1', 'sentence2']),
}

task_config = task_configs[task_param]

In [ ]:
if task_param == "mnli_matched": 
    data = load_dataset("glue", "mnli") 
    val_key = "validation_matched"
    test_key = "test_matched"
elif task_param == "mnli_mismatched":
    data = load_dataset("glue", "mnli") 
    val_key = "validation_mismatched"
    test_key = "test_mismatched"
else:
    data = load_dataset("glue", task_param)
    val_key = "validation"
    test_key = "test"

data

In [ ]:
X_train = data["train"]
y = "label"
X_val = data[val_key]

In [ ]:
adapter_param_grid = {
    'num_epochs': [50],
    'batch_size': [32, 512],
    'learning_rate': [1e-2, 1e-3],
    'category': [task_config.class_type],
    'device': [device_name]
}

# Create a list of all combinations of hyperparameters
adapter_params = [dict(zip(adapter_param_grid.keys(), v)) for v in itertools.product(*adapter_param_grid.values())]
print(f"{len(adapter_params)} hyperparameter combinations")

In [ ]:
# Setup for saving power outputs
results_folder = pathlib.Path(f"results/adapter/{task_param}")
results_folder.mkdir(parents=True, exist_ok=True)
save_file_id = datetime.now().strftime("%Y%m%d_%H%M%S")
results_file = results_folder / f"val_{save_file_id}.csv"
with open(results_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['power', 'time'])
print(f"saving results to ./{results_file}")

# Iterate over all combinations of hyperparameters
adapter_bar = tqdm(enumerate(adapter_params), total=len(adapter_params))
for i, params in adapter_bar:
    # Formatting params to display
    print_params = params.copy()
    for param in ['category', 'device']:
        del print_params[param]

    # Initialize the model with current set of hyperparameters
    adapter = BERTAdapter(**params)

    # Create a thread for tracking power output (NVIDIA ONLY), logs power and time
    def get_power():
        pynvml.nvmlInit()
        while not adapter.train_stop_flag:
            handle = pynvml.nvmlDeviceGetHandleByIndex(0)  # Returns device handle, index 0 specifies the first GPU
            power = pynvml.nvmlDeviceGetPowerUsage(handle)  # Retrieves power usage for GPU in milliwatts
            power = power / 1000  # convert mW to W
            curr_time = time.time()
            print("Current NVIDIA power output:", power, "at time", curr_time)
            # Write to results csv
            with open(results_file, 'a', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow([power, curr_time])
                
        pynvml.nvmlShutdown()
        
    power_thread = threading.Thread(target=get_power)

    # Make sure power output was recorded in the correct time range
    print("Start time:", time.time()) 

    power_thread.start()  # Start the benchmarking thread
    metrics, train_times_per_epoch, energy_per_epoch = adapter.fit(X_train, y, X_val, y)
    power_thread.join()  # Wait for the thread to complete
        
    print("End time:", time.time())